In [1]:
import argparse
import os
import sys
import logging
import time
import numpy as np
import torch
import torch.utils.data
import torchvision
import open3d as o3d
from scipy.spatial.transform import Rotation
from sklearn.neighbors import KDTree

sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(os.path.realpath('__file__')), os.pardir)))
import ptlk

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def options(argv=None):
    parser = argparse.ArgumentParser(description='PointNet-LK')

    parser.add_argument('-i', '--dataset-path', default='D:/SDXX/ModelNet40', type=str,
                        metavar='PATH', help='path to the input dataset') # like 'modelnet40_half2'
    parser.add_argument('-c', '--categoryfile', default='./sampledata/different_test.txt', type=str,
                        metavar='PATH', help='path to the categories to be tested') # eg. './sampledata/modelnet40_half1.txt'
    parser.add_argument('-p', '--perturbations', default='./perturbations/different/pertur_40.csv', type=str,
                        metavar='PATH', help='path to the perturbation file') # see. generate_perturbations.py

    # settings for input data
    parser.add_argument('--dataset-type', default='modelnet', choices=['modelnet'],
                        metavar='DATASET', help='dataset type (default: modelnet)')
    parser.add_argument('--format', default='wv', choices=['wv', 'wt'],
                        help='perturbation format (default: wv (twist)) (wt: rotation and translation)') 

    # settings for PointNet-LK
    parser.add_argument('--max-iter', default=30, type=int,
                        metavar='N', help='max-iter on LK. (default: 20)')
    parser.add_argument('--pretrained', default='model_best_model_best.pth', type=str,    #
                        metavar='PATH', help='path to trained model file (default: null (no-use))')
    parser.add_argument('--transfer-from', default='', type=str,
                        metavar='PATH', help='path to classifier feature (default: null (no-use))')
    parser.add_argument('--dim-k', default=1024, type=int,
                        metavar='K', help='dim. of the feature vector (default: 1024)')
    parser.add_argument('--symfn', default='max', choices=['max', 'avg'],
                        help='symmetric function (default: max)')
    parser.add_argument('--delta', default=1.0e-2, type=float,
                        metavar='D', help='step size for approx. Jacobian (default: 1.0e-2)')

    # settings for on testing
    parser.add_argument('-l', '--logfile', default='', type=str,
                        metavar='LOGNAME', help='path to logfile (default: null (no logging))')
    parser.add_argument('-j', '--workers', default=4, type=int,
                        metavar='N', help='number of data loading workers (default: 4)')
    parser.add_argument('--device', default='cpu', type=str,
                        metavar='DEVICE', help='use CUDA if available (default: cpu)')

    args = parser.parse_args(argv)
    return args

In [3]:
class Action:
    def __init__(self, args):
        # PointNet
        self.transfer_from = args.transfer_from
        self.dim_k = args.dim_k
        self.sym_fn = None
        if args.symfn == 'max':
            self.sym_fn = ptlk.pointnet.symfn_max
        elif args.symfn == 'avg':
            self.sym_fn = ptlk.pointnet.symfn_avg
        # LK
        self.delta = args.delta
        self.max_iter = args.max_iter
        self.xtol = 1.0e-7
        self.p0_zero_mean = True
        self.p1_zero_mean = True

    def create_model(self):
        ptnet = self.create_pointnet_features()
        return self.create_from_pointnet_features(ptnet)

    def create_pointnet_features(self):
        ptnet = ptlk.pointnet.PointNet_features(self.dim_k, use_tnet=False, sym_fn=self.sym_fn)
        if self.transfer_from and os.path.isfile(self.transfer_from):
            ptnet.load_state_dict(torch.load(self.transfer_from, map_location='cpu'))
        return ptnet

    def create_from_pointnet_features(self, ptnet):
        return ptlk.pointlk.PointLK(ptnet, self.delta)
    
    def eval_1(self, model, device):
        model.eval()
        p0, p1 = self.read_ply(self.templatefile, self.sourcefile)
        p0 = torch.tensor(p0).unsqueeze(0)
        p1 = torch.tensor(p1).unsqueeze(0)
#        p0, p1 = data_tensor
        res = self.do_estimate(p0, p1, model, device)
        g_hat = res.cpu().contiguous().view(-1, 4, 4)
        est_trans_p1 = ptlk.se3.transform(g_hat.to(p0), p1)
        
        return est_trans_p1
#                   
    def do_estimate(self, p0, p1, model, device):
        p0 = p0.to(device) # template (1, N, 3)
        p1 = p1.to(device) # source (1, M, 3)
        r  = ptlk.pointlk.PointLK.do_forward(model, p0, p1, self.max_iter, self.xtol,\
                                            self.p0_zero_mean, self.p1_zero_mean)

        #r = model(p0, p1, self.max_iter)
        est_g = model.g # (1, 4, 4)

        return est_g, r

In [4]:
args = options([])
act = Action(args)
model = act.create_model()
if args.pretrained:
    assert os.path.isfile(args.pretrained)
    model.load_state_dict(torch.load(args.pretrained, map_location='cpu'))
model.to(args.device)
model.eval()

PointLK(
  (ptnet): PointNet_features(
    (h1): Sequential(
      (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
    (h2): Sequential(
      (0): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (7): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU()
    )
  )
)

# Bunny

In [5]:
#sol = np.array([2.20761e-05, -3.34606e-05, -7.20881e-05, 0.000335889, -0.708202, 0.000602459, 0.706009])     #90
sol = np.array([-0.0520211, -0.000383981, -0.0109223, 0.00548449, -0.294635, -0.0038555, 0.955586])         #45
r = Rotation.from_quat(sol[3:])
rot = r.as_matrix()
trans = sol[:3]
G = np.concatenate((rot.T, trans.reshape(3, -1)), axis = 1)
G = np.concatenate((G, [[0.,0.,0.,1.]]), axis = 0)

In [6]:
bun_pt = o3d.io.read_point_cloud(r'C:\Users\fcky\Desktop\3D\bunny\bunny\data\bun000.ply')
bun_pt.paint_uniform_color([0.5, 0.5, 0.5])
bun_pt_tree = o3d.geometry.KDTreeFlann(bun_pt)
bun_ps = o3d.io.read_point_cloud(r'C:\Users\fcky\Desktop\3D\bunny\bunny\data\bun045.ply')
bun_ps.paint_uniform_color([0.5, 0.5, 0.5])

PointCloud with 40097 points.

In [7]:
tgt = np.asarray(bun_pt.points)
scr = np.asarray(bun_ps.points)

In [8]:
#noise
def jitter_pointcloud(pointcloud, sigma=0.001, clip=0.05):
    N, C = pointcloud.shape
    pointcloud += np.clip(sigma * np.random.randn(N, C), -1 * clip, clip)
    return pointcloud
tgt = jitter_pointcloud(tgt, sigma=0.001)
scr = jitter_pointcloud(scr, sigma=0.001)

In [8]:
tgt_sorted = np.array(sorted(tgt.tolist(), key=lambda x:(x[0], x[1])))
scr_sorted = np.array(sorted(scr.tolist(), key=lambda x:(x[0], x[1])))

In [9]:
pt = torch.tensor(tgt_sorted).unsqueeze(0).contiguous().to(torch.float32)
ps = torch.tensor(scr_sorted).unsqueeze(0).contiguous().to(torch.float32)

In [10]:
# 原图像
bun_pt.paint_uniform_color([255/255, 127/255, 39/255])
bun_pt.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
bun_pt.orient_normals_to_align_with_direction()
bun_ps.paint_uniform_color([160/255, 255/255, 249/255])
bun_ps.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
bun_ps.orient_normals_to_align_with_direction()
o3d.visualization.draw_geometries([bun_pt, bun_ps])

In [11]:
bun_ps.transform(G)
bun_ps_tree = o3d.geometry.KDTreeFlann(bun_ps)

In [21]:
count = 0
idxs_pt = []
idxs_ps = []
for i in range(len(bun_ps.points)):
    #[k, idx, dist] = bun00_tree.search_knn_vector_3d(bun45.points[int(ind1[i])], 3)
    [k, idx, dist] = bun_pt_tree.search_radius_vector_3d(bun_ps.points[i], 0.0005)
    if k!= 0:
        count += 1
        idxs_pt.append(idx[0])
        idxs_ps.append(i)
count/len(bun_ps.points)

0.3504065308272162

In [11]:
np.asarray(bun_pt.colors)[idxs_pt[0:], :] = [255/255, 127/255, 39/255]
np.asarray(bun_ps.colors)[idxs_ps[0:], :] = [160/255, 255/255, 249/255]
o3d.visualization.draw_geometries([bun_pt, bun_ps])

In [12]:
tgtidx = torch.tensor(np.random.randint(0,tgt.shape[0],1024), dtype = torch.long)
scridx = torch.tensor(np.random.randint(0,scr.shape[0],1024), dtype = torch.long)

In [26]:
pt_kwy = int(tgt.shape[0] / 1024)
pt_num = np.random.randint(0, pt_kwy, 1)
tgtidx = list(range(pt_num[0], tgt.shape[0], int(pt_kwy+1)))
ps_kwy = int(scr.shape[0] / 1024)
ps_num = np.random.randint(0, ps_kwy, 1)
scridx = list(range(ps_num[0], scr.shape[0], int(ps_kwy+1)))

In [27]:
sampt = pt[:,tgtidx,:]
samps = ps[:,scridx,:]
with torch.no_grad():
    since = time.time()
    bu_est_g, bu_r = act.do_estimate(sampt, samps, model, args.device)
    time_elapsed = time.time() - since

In [17]:
spend_time = []
error_Gs_ran = []
rot_ran = []
trans_ran = []
for i in range(10):
    tgtidx = torch.tensor(np.random.randint(0,tgt.shape[0],2048), dtype = torch.long)
    scridx = torch.tensor(np.random.randint(0,scr.shape[0],2048), dtype = torch.long)
    sampt = pt[:,tgtidx,:]
    samps = ps[:,scridx,:]
    with torch.no_grad():
        since = time.time()
        est_g, bu_r = act.do_estimate(sampt, samps, model, args.device)
        time_elapsed = time.time() - since
        
        est_g = est_g.reshape(4, 4).numpy()
        test_g_mse = np.mean((G - est_g) ** 2)
        test_g_rmse = np.sqrt(test_g_mse)
        
        rot_ran.append(G[0:3, 0:3] - est_g[0:3, 0:3])
        trans_ran.append(G[0:3, 3] - est_g[0:3, 3])       
        error_Gs_ran.append(test_g_rmse)
        spend_time.append(time_elapsed)

In [18]:
avg_g_10 = np.mean(np.array(error_Gs_ran))
print( 'time_elapsed: %.4f' % avg_g_10)

time_elapsed: 0.0352


In [19]:
avg_times_10 = np.mean(np.array(spend_time))
print( 'time_elapsed: %.4f' % avg_times_10)

time_elapsed: 1.0462


In [22]:
#no sampling
with torch.no_grad():
    since = time.time()
    bu_est_g, bu_r = act.do_estimate(pt, ps, model, args.device)
    time_elapsed = time.time() - since

### 误差计算

In [28]:
est_rot = bu_est_g[:, 0:3, 0:3].reshape(3, 3).numpy()
est_trans = bu_est_g[:, 0:3, 3].reshape(3, 1).numpy()
gt_rot = G[0:3, 0:3]
gt_trans = G[0:3, 3].reshape(3, 1)

In [29]:
test_r_mse = np.mean((gt_rot - est_rot) ** 2)
test_r_rmse = np.sqrt(test_r_mse)
test_r_mae = np.mean(np.abs(gt_rot - est_rot))
test_t_mse = np.mean((gt_trans - est_trans) ** 2)
test_t_rmse = np.sqrt(test_t_mse)
test_t_mae = np.mean(np.abs(gt_trans - est_trans))

In [20]:
test_r_mse = np.mean(np.array(rot_ran) ** 2)
test_r_rmse = np.sqrt(test_r_mse)
test_r_mae = np.mean(np.abs(np.array(rot_ran)))
test_t_mse = np.mean(np.array(trans_ran) ** 2)
test_t_rmse = np.sqrt(test_t_mse)
test_t_mae = np.mean(np.abs(np.array(trans_ran)))

NameError: name 'rot_ran' is not defined

In [30]:
message_r = 'test_r_mse:{:.6f},test_r_rmse:{:.6f},test_r_mae:{:.6f}'.format(test_r_mse, test_r_rmse, test_r_mae)
message_t = 'test_t_mse:{:.6f},test_t_rmse:{:.6f},test_t_mae:{:.6f}'.format(test_t_mse, test_t_rmse, test_t_mae)
print(message_r, message_t)

test_r_mse:0.001737,test_r_rmse:0.041677,test_r_mae:0.032711 test_t_mse:0.000003,test_t_rmse:0.001610,test_t_mae:0.001224


### 图像

In [14]:
g_series = model.g_series

In [31]:
g_hat = bu_est_g.cpu().contiguous().view(-1, 4, 4)
est_trans_pc0 = ptlk.se3.transform(g_hat.to(ps), ps.transpose(1,2).contiguous()).transpose(1,2).contiguous()
est_pc00 = est_trans_pc0[0].detach().numpy()
est_pc00.shape

(40097, 3)

In [32]:
est_bupt = o3d.geometry.PointCloud()
est_bupt.points = o3d.utility.Vector3dVector(est_pc00.reshape(-1, 3))
est_bupt.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
est_bupt.orient_normals_to_align_with_direction()
#est_ptpcd.paint_uniform_color([0.5, 0.5, 0.5])
#np.asarray(est_ptpcd.colors)[scridx[0:], :] = [160/255, 255/255, 249/255]
est_bupt.paint_uniform_color([160/255, 255/255, 249/255])

PointCloud with 40097 points.

In [33]:
o3d.visualization.draw_geometries([bun_pt, est_bupt])

In [22]:
# writing files
with open(r'est_pt.txt', 'w') as fout:
    for i in range(est_pc00.shape[0]):
        print(' '.join(map(str, est_pc00.reshape(-1, 3)[i])), file=fout)
with open(r'pt.txt', 'w') as fout:
    for i in range(tgt.shape[0]):
        print(' '.join(map(str, tgt.reshape(-1, 3)[i])), file=fout)

In [20]:
centroid_pt = np.mean(tgt, axis=0).reshape(1,3)
centroid_ps = np.mean(est_pc00, axis=0).reshape(1,3)
centroid = (centroid_ps+centroid_pt)/2

In [21]:
pts = np.concatenate((tgt, centroid), axis = 0)
pss = np.concatenate((est_pc00, centroid), axis = 0)
bun_pt.points = o3d.utility.Vector3dVector(pts.reshape(-1, 3))
bun_ps.points = o3d.utility.Vector3dVector(pss.reshape(-1, 3))

In [22]:
bun_pt.paint_uniform_color([0.5, 0.5, 0.5])
bun_ps.paint_uniform_color([0.5, 0.5, 0.5])
bun_pt.colors[-1] = [255/255, 127/255, 39/255]
bun_ps.colors[-1] = [160/255, 255/255, 249/255]
o3d.visualization.draw_geometries([bun_pt, bun_ps])

In [26]:
est_bupt_tree = o3d.geometry.KDTreeFlann(est_bupt)
ptnet = model.ptnet

In [93]:
re_tgtidx = torch.tensor(np.random.randint(0,tgt.shape[0],100), dtype = torch.long)
re_scridx = torch.tensor(np.random.randint(0,scr.shape[0],100), dtype = torch.long)
re_pt = pt[:,re_tgtidx,:]
re_ps = est_trans_pc0[:,re_scridx,:]

In [107]:
re_idxs =[]
pt_indexs = []
new_ptfeas = torch.zeros(100, 1024)
for i in range(re_pt.shape[1]):
    #[k, idx, dist] = bun_pt_tree.search_knn_vector_3d(re_pt[0][i].reshape(3, 1), 256)
    [k, idx, dist] = bun_pt_tree.search_radius_vector_3d(re_pt[0][i].reshape(3, 1), 0.01)
    t_pt = pt[:,idx,:]
    new_ptfeas[i] =  ptnet(t_pt).reshape(1024)
    pt_indexs.append(idx)
    for j in range(len(idx)):
        re_idxs.append(idx[j])
#    break

In [130]:
bun_pt.paint_uniform_color([0.5, 0.5, 0.5])
np.asarray(bun_pt.colors)[re_idxs[:], :] = [255/255, 127/255, 39/255]
o3d.visualization.draw_geometries([bun_pt])

In [109]:
re_psidxs =[]
ps_indexs = []
new_psfeas = torch.zeros(100, 1024)
for i in range(re_ps.shape[1]):
    #[k, idx, dist] = est_bupt_tree.search_knn_vector_3d(re_ps[0][i].reshape(3, 1), 256)
    [k, idx, dist] = est_bupt_tree.search_radius_vector_3d(re_ps[0][i].reshape(3, 1), 0.01)
    t_ps = est_trans_pc0[:,idx,:]
    new_psfeas[i] =  ptnet(t_ps).reshape(1024)
    ps_indexs.append(idx)
    for j in range(len(idx)):
        re_psidxs.append(idx[j])
#    break

In [129]:
est_bupt.paint_uniform_color([0.5, 0.5, 0.5])
np.asarray(est_bupt.colors)[re_psidxs[:], :] = [255/255, 127/255, 39/255]
o3d.visualization.draw_geometries([est_bupt])

In [111]:
#pt_feas, ps_feas = bu_feas['pt_feas'], bu_feas['ps_feas']
ptfea_tree = KDTree(new_ptfeas.detach().numpy())

In [112]:
correspond = []
dists = []
#K = 3
for i, fea in enumerate(new_psfeas):
    #idx = ptfea_tree.query_radius(fea.detach().numpy().reshape(1, -1), r=1)
    dist, idx = ptfea_tree.query(fea.detach().numpy().reshape(1, -1), k=1) 
    # = tup[0], tup[1]
    dists.append((i, dist, idx))
#    for j in range(idx[0].shape[-1]):
#        correspond.append([i, idx[0][j]])  #, dist[0][j]])

In [50]:
correspondences = np.array(correspond)
ps_idx = list(set(correspondences[:,0].tolist()))
pt_idx = list(set(correspondences[:,1].tolist()))
len(pt_idx), len(ps_idx)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [76]:
bun_pt.paint_uniform_color([0.5, 0.5, 0.5])
est_bupt.paint_uniform_color([0.5, 0.5, 0.5])
np.asarray(bun_pt.colors)[pt_idx[:], :] = [1, 0, 0]
np.asarray(est_bupt.colors)[ps_idx[:], :] = [0, 0, 1]
o3d.visualization.draw_geometries([bun_pt, est_bupt])

In [22]:
esttgt = np.asarray(est_bupt.points)
centroid_scr = np.mean(esttgt, axis=0).reshape(1,3)
centroid_tgt = np.mean(tgt, axis=0).reshape(1,3)
scrpoints = np.concatenate((esttgt, centroid_scr), axis = 0)
tgtpoints = np.concatenate((tgt, centroid_tgt), axis = 0)
est_bupt.points = o3d.utility.Vector3dVector(scrpoints.reshape(-1, 3))
bun_pt.points = o3d.utility.Vector3dVector(tgtpoints.reshape(-1, 3))

In [24]:
bun_pt.paint_uniform_color([0.5, 0.5, 0.5])
est_bupt.paint_uniform_color([0.5, 0.5, 0.5])
bun_pt.colors[-1] = [1, 0, 0]
est_bupt.colors[-1] = [0, 0, 1]
o3d.visualization.draw_geometries([bun_pt, est_bupt])

In [74]:
sam_tgt = tgt[pt_idx[1], :]
sam_tgt.reshape(1,3)
#[k, idx, dist] = bun_pt_tree.search_knn_vector_3d(sam_tgt.reshape(3,1), 100)
[k, ptid, dist] = bun_pt_tree.search_radius_vector_3d(sam_tgt.reshape(3,1), 0.01)

In [56]:
sam_tgt = tgt[pt_idx[1], :]
idxs_pt = []
for i in range(sam_tgt.shape[0]):
    #[k, idx, dist] = bun_pt_tree.search_knn_vector_3d(sam_tgt[i].reshape(1,3), 100)
    [k, idx, dist] = bun_pt_tree.search_radius_vector_3d(sam_tgt[i], 0.01)
    for j in range(len(idx)):
        idxs_pt.append(idx[j])

ValueError: cannot reshape array of size 1 into shape (1,3)

In [70]:
bun_pt.paint_uniform_color([0.5, 0.5, 0.5])
np.asarray(bun_pt.colors)[idx[0:], :] = [255/255, 127/255, 39/255]
o3d.visualization.draw_geometries([bun_pt])

In [75]:
sam_scr = scr[ps_idx[1], :]
#[k, idx, dist] = bun_pt_tree.search_knn_vector_3d(sam_tgt.reshape(3,1), 100)
[k, psid, dist] = bun_ps_tree.search_radius_vector_3d(sam_scr.reshape(3,1), 0.01)

In [48]:
sam_scr = scr[ps_idx, :]
idxs_ps = []
for i in range(sam_scr.shape[0]):
    #[k, idx, dist] = bun_ps_tree.search_knn_vector_3d(sam_scr[i], 100)
    [k, idx, dist] = bun_ps_tree.search_radius_vector_3d(sam_scr[i], 0.01)
    for j in range(len(idx)):
        idxs_ps.append(idx[j]) 

In [72]:
bun_ps.paint_uniform_color([0.5, 0.5, 0.5])
np.asarray(bun_ps.colors)[idxs_ps[0:], :] = [160/255, 255/255, 249/255]
o3d.visualization.draw_geometries([bun_pt, bun_ps])

In [76]:
tgtidx = torch.tensor(np.random.randint(0,tgt.shape[0],1024), dtype = torch.long)
scridx = torch.tensor(np.random.randint(0,scr.shape[0],1024), dtype = torch.long)
fi_pt = pt[:,ptid,:]
fi_ps = est_trans_pc0[:,psid,:]

In [36]:
with torch.no_grad():
    fi_est_g, fi_r, fi_f_series, fi_feas = act.do_estimate(est_trans_pc0, pt, model, args.device)

In [79]:
with torch.no_grad():
    fi_est_g, fi_r, fi_f_series, fi_feas = act.do_estimate(fi_pt, fi_ps, model, args.device)

# dragonStand

In [21]:
sol = np.array([-0.0010015778, -3.7185848e-05, -0.00049660774, 0.00065300596, -0.40700561, 0.0014209391, 0.91342431])         #48
#sol = np.array([-0.0011086911, -0.0001244843, 9.4951596e-05, 0.0009286101, -0.58797526, 0.0021693073, 0.8088755])         #72
r = Rotation.from_quat(sol[3:])
rot = r.as_matrix()
trans = sol[:3]
G = np.concatenate((rot.T, trans.reshape(3, -1)), axis = 1)
G = np.concatenate((G, [[0.,0.,0.,1.]]), axis = 0)

In [22]:
r.as_euler('zyx', degrees=True)

array([  0.26796503, -48.03348845,   0.20132109])

In [6]:
DS_pt = o3d.io.read_point_cloud(r'C:\Users\fcky\Desktop\3D\dragon_stand\dragonStandRight_0.ply')
DS_ps = o3d.io.read_point_cloud(r'C:\Users\fcky\Desktop\3D\dragon_stand\dragonStandRight_48.ply')
tgt = np.asarray(DS_pt.points)
scr = np.asarray(DS_ps.points)
tgt_sorted = np.array(sorted(tgt.tolist(), key=lambda x:(x[0], x[1])))
scr_sorted = np.array(sorted(scr.tolist(), key=lambda x:(x[0], x[1])))
pt = torch.tensor(tgt_sorted).unsqueeze(0).contiguous().to(torch.float32)
ps = torch.tensor(scr_sorted).unsqueeze(0).contiguous().to(torch.float32)
DS_pt_tree = o3d.geometry.KDTreeFlann(DS_pt)

In [7]:
# 原图像
DS_pt.paint_uniform_color([255/255, 127/255, 39/255])
DS_pt.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
DS_pt.orient_normals_to_align_with_direction()
DS_ps.paint_uniform_color([160/255, 255/255, 249/255])
DS_ps.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
DS_ps.orient_normals_to_align_with_direction()
o3d.visualization.draw_geometries([DS_pt, DS_ps])

In [8]:
DS_ps.transform(G)
count = 0
idxs_pt = []
idxs_ps = []
for i in range(len(DS_ps.points)):
    #[k, idx, dist] = HS_pt_tree.search_knn_vector_3d(HS_ps.points[i], 1)
    [k, idx, dist] = DS_pt_tree.search_radius_vector_3d(DS_ps.points[i], 0.005)
    if k!= 0:
        count += 1
        idxs_pt.append(idx[0])
        idxs_ps.append(i)
count/len(DS_ps.points)
#dist

0.8585008147745791

In [8]:
tgtidx = torch.tensor(np.random.randint(0,tgt.shape[0],1024), dtype = torch.long)
scridx = torch.tensor(np.random.randint(0,scr.shape[0],1024), dtype = torch.long)

In [17]:
pt_kwy = int(tgt.shape[0] / 1024)
pt_num = np.random.randint(0, pt_kwy, 1)
tgtidx = list(range(pt_num[0], tgt.shape[0], int(pt_kwy+1)))
ps_kwy = int(scr.shape[0] / 1024)
ps_num = np.random.randint(0, ps_kwy, 1)
scridx = list(range(ps_num[0], scr.shape[0], int(ps_kwy+1)))

In [9]:
#sampling
sampt = pt[:,tgtidx,:]
samps = ps[:,scridx,:]
with torch.no_grad():
    est_g, r = act.do_estimate(sampt, samps, model, args.device)

In [43]:
#no sampling
with torch.no_grad():
    est_g, ds_r = act.do_estimate(pt, ps, model, args.device)

In [10]:
g_hat = est_g.cpu().contiguous().view(-1, 4, 4)
est_trans_pt = ptlk.se3.transform(g_hat.to(ps), ps.transpose(1,2).contiguous()).transpose(1,2).contiguous()
est_pt = est_trans_pt[0].detach().numpy()
est_pt.shape

(22092, 3)

In [11]:
est_DSpt = o3d.geometry.PointCloud()
est_DSpt.points = o3d.utility.Vector3dVector(est_pt.reshape(-1, 3))
est_DSpt.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
est_DSpt.orient_normals_to_align_with_direction()
est_DSpt.paint_uniform_color([160/255, 255/255, 249/255])

PointCloud with 22092 points.

In [12]:
o3d.visualization.draw_geometries([DS_pt, est_DSpt])

### error

In [13]:
est_rot = est_g[:, 0:3, 0:3].reshape(3, 3).numpy()
est_trans = est_g[:, 0:3, 3].reshape(3, 1).numpy()
gt_rot = G[0:3, 0:3]
gt_trans = G[0:3, 3].reshape(3, 1)

In [14]:
test_r_mse = np.mean((gt_rot - est_rot) ** 2)
test_r_rmse = np.sqrt(test_r_mse)
test_r_mae = np.mean(np.abs(gt_rot - est_rot))
test_t_mse = np.mean((gt_trans - est_trans) ** 2)
test_t_rmse = np.sqrt(test_t_mse)
test_t_mae = np.mean(np.abs(gt_trans - est_trans))

In [15]:
message_r = 'test_r_mse:{:.6f},test_r_rmse:{:.6f},test_r_mae:{:.6f}'.format(test_r_mse, test_r_rmse, test_r_mae)
message_t = 'test_t_mse:{:.6f},test_t_rmse:{:.6f},test_t_mae:{:.6f}'.format(test_t_mse, test_t_rmse, test_t_mae)
print(message_r, message_t)

test_r_mse:0.000222,test_r_rmse:0.014883,test_r_mae:0.011403 test_t_mse:0.000007,test_t_rmse:0.002725,test_t_mae:0.002415


In [62]:
message_r = 'test_r_mse:{:.6f},test_r_rmse:{:.6f},test_r_mae:{:.6f}'.format(test_r_mse, test_r_rmse, test_r_mae)
message_t = 'test_t_mse:{:.6f},test_t_rmse:{:.6f},test_t_mae:{:.6f}'.format(test_t_mse, test_t_rmse, test_t_mae)
print(message_r, message_t)

test_r_mse:0.000354,test_r_rmse:0.018825,test_r_mae:0.016658 test_t_mse:0.000001,test_t_rmse:0.000927,test_t_mae:0.000762


In [16]:
# writing files
fname_t = r'C:\Users\fcky\Desktop\PointNetLK-master1\results/pt.txt'
fname_s = r'C:\Users\fcky\Desktop\PointNetLK-master1\results/est_pt.txt'
with open(fname_s, 'w') as fout:
    for i in range(est_pt.shape[0]):
        print(' '.join(map(str, est_pt.reshape(-1, 3)[i])), file=fout)
with open(fname_t, 'w') as fout:
    for i in range(tgt.shape[0]):
        print(' '.join(map(str, tgt.reshape(-1, 3)[i])), file=fout)

# Bird

In [5]:
def readasc(fpath):
    with open(fpath, 'r') as fin:
        pcd = []
        sig = fin.readline().strip()
        for v in range(int(sig)):
            vp = tuple(float(s) for s in fin.readline().strip().split('\t'))
            pcd.append(vp)
    pcd0 = np.array(pcd)
    return pcd0

def unit(pcd):
    c = np.max(pcd, axis = 0) - np.min(pcd, axis = 0)
    s = np.max(c)
    v = pcd / s
    return v - v.mean(axis = 0)

In [54]:
tgt = readasc(r'C:\Users\fcky\Desktop\3D\DaiMa\EBABC_Kuang\angel-deg0.asc')
scr = readasc(r'C:\Users\fcky\Desktop\3D\DaiMa\EBABC_Kuang\angel-deg40.asc')
tgt = unit(tgt)
scr = unit(scr)

In [55]:
ptpcd = o3d.geometry.PointCloud()
ptpcd.points = o3d.utility.Vector3dVector(tgt.reshape(-1, 3))
ptpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
ptpcd.orient_normals_to_align_with_direction()
ptpcd.paint_uniform_color([255/255, 127/255, 39/255])

pspcd = o3d.geometry.PointCloud()
pspcd.points = o3d.utility.Vector3dVector(scr.reshape(-1, 3))
pspcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
pspcd.orient_normals_to_align_with_direction()
pspcd.paint_uniform_color([160/255, 255/255, 249/255])

o3d.visualization.draw_geometries([ptpcd, pspcd]) 

In [14]:
r = Rotation.from_euler('zyx', [5, 40, 0], degrees=True)
rot = r.as_matrix()
trans = np.array([0.,0.,0.])
G = np.concatenate((rot.T, trans.reshape(3, -1)), axis = 1)
G = np.concatenate((G, [[0.,0.,0.,1.]]), axis = 0)

In [15]:
pspcd.transform(G)
o3d.visualization.draw_geometries([ptpcd, pspcd]) 

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


In [ ]:
count = 0
idxs_pt = []
idxs_ps = []
for i in range(len(DS_ps.points)):
    #[k, idx, dist] = HS_pt_tree.search_knn_vector_3d(HS_ps.points[i], 1)
    [k, idx, dist] = DS_pt_tree.search_radius_vector_3d(DS_ps.points[i], 0.005)
    if k!= 0:
        count += 1
        idxs_pt.append(idx[0])
        idxs_ps.append(i)
count/len(DS_ps.points)
#dist

In [80]:
tgt_sorted = np.array(sorted(tgt.tolist(), key=lambda x:(x[0], x[1])))
scr_sorted = np.array(sorted(scr.tolist(), key=lambda x:(x[0], x[1])))
pt = torch.tensor(tgt_sorted).unsqueeze(0).contiguous().to(torch.float32)
ps = torch.tensor(scr_sorted).unsqueeze(0).contiguous().to(torch.float32)

In [81]:
tgtidx = torch.tensor(np.random.randint(0,tgt.shape[0],1024), dtype = torch.long)
scridx = torch.tensor(np.random.randint(0,scr.shape[0],1024), dtype = torch.long)
sampt = pt[:,tgtidx,:]
samps = ps[:,scridx,:]

In [82]:
#no sampling
with torch.no_grad():
    est_g, r = act.do_estimate(sampt, samps, model, args.device)

In [83]:
g_hat = est_g.cpu().contiguous().view(-1, 4, 4)
est_trans_pt = ptlk.se3.transform(g_hat.to(ps), ps.transpose(1,2).contiguous()).transpose(1,2).contiguous()
est_pt = est_trans_pt[0].detach().numpy()
est_pt.shape

(13190, 3)

In [84]:
est_ptpcd = o3d.geometry.PointCloud()
est_ptpcd.points = o3d.utility.Vector3dVector(est_pt.reshape(-1, 3))
est_ptpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
est_ptpcd.orient_normals_to_align_with_direction()
est_ptpcd.paint_uniform_color([160/255, 255/255, 249/255])

PointCloud with 13190 points.

In [85]:
o3d.visualization.draw_geometries([ptpcd, est_ptpcd])

In [83]:
def write(fname_tem, fname_est, pc_tem, pc_est):
    with open(fname_est, 'w') as fout:
        for i in range(pc_est.shape[0]):
            print(' '.join(map(str, pc_est[i]))+' '+'80 167 73',file=fout)   #变换后的源：绿色
            
    with open(fname_tem, 'w') as fout:
        for i in range(pc_tem.shape[0]):
            print(' '.join(map(str, pc_tem[i]))+' '+'243 223 131',file=fout)  #模板：黄色

In [84]:
fname_t = r'C:\Users\fcky\Desktop\PointNetLK-master1\results/pt.txt'
fname_s = r'C:\Users\fcky\Desktop\PointNetLK-master1\results/est_pt.txt'

write(fname_t, fname_s, tgt, est_pt)

In [86]:
#angel
G = np.array([[0.764735579491, 0.148309335113, -0.627043724060, -0.082956433296],
            [-0.152645140886, 0.987147688866, 0.047317363322, 0.013409459963],
            [0.626002371311, 0.059529908001, 0.777545630932, 0.008217109367],
            [0.000000000000, 0.000000000000, 0.000000000000, 1.000000000000]])

In [22]:
#tele
G = np.array([[0.807809889317, 0.207861557603, -0.551576614380, -0.066947236657],
              [-0.198102071881, 0.977066755295, 0.078077659011, 0.005881545600],
              [0.555156469345, 0.046196557581, 0.830461978912, 0.006951204035],
              [0.000000000000, 0.000000000000, 0.000000000000, 1.000000000000]])

In [87]:
est_rot = est_g[:, 0:3, 0:3].reshape(3, 3).numpy()
est_trans = est_g[:, 0:3, 3].reshape(3, 1).numpy()
gt_rot = G[0:3, 0:3]
gt_trans = G[0:3, 3].reshape(3, 1)

In [88]:
test_r_mse = np.mean((gt_rot - est_rot) ** 2)
test_r_rmse = np.sqrt(test_r_mse)
test_r_mae = np.mean(np.abs(gt_rot - est_rot))
test_t_mse = np.mean((gt_trans - est_trans) ** 2)
test_t_rmse = np.sqrt(test_t_mse)
test_t_mae = np.mean(np.abs(gt_trans - est_trans))

In [89]:
message_r = 'test_r_mse:{:.4f},test_r_rmse:{:.4f},test_r_mae:{:.4f}'.format(test_r_mse, test_r_rmse, test_r_mae)
message_t = 'test_t_mse:{:.4f},test_t_rmse:{:.4f},test_t_mae:{:.4f}'.format(test_t_mse, test_t_rmse, test_t_mae)
print(message_r, message_t)

test_r_mse:0.0006,test_r_rmse:0.0235,test_r_mae:0.0204 test_t_mse:0.0001,test_t_rmse:0.0077,test_t_mae:0.0075


# ModelNet40

In [5]:
import h5py
import glob

In [6]:
def load_data(partition):
    BASE_DIR = r'D:\SDXX\data'
    all_data = []
    all_label = []
    for h5_name in glob.glob(os.path.join(BASE_DIR, 'modelnet40_ply_hdf5_2048', 'ply_data_%s*.h5' % partition)):
        f = h5py.File(h5_name)
        data = f['data'][:].astype('float32')
        label = f['label'][:].astype('int64')
        f.close()
        all_data.append(data)
        all_label.append(label)
    all_data = np.concatenate(all_data, axis=0)
    all_label = np.concatenate(all_label, axis=0)
    return all_data, all_label

def jitter_pointcloud(pointcloud, sigma=0.01, clip=0.05):
    N, C = pointcloud.shape
    pointcloud += np.clip(sigma * np.random.randn(N, C), -1 * clip, clip)
    return pointcloud

In [7]:
def uniform_2_sphere(num: int = None):
    if num is not None:
        phi = np.random.uniform(0.0, 2 * np.pi, num)
        cos_theta = np.random.uniform(-1.0, 1.0, num)
    else:
        phi = np.random.uniform(0.0, 2 * np.pi)
        cos_theta = np.random.uniform(-1.0, 1.0)

    theta = np.arccos(cos_theta)
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)

    return np.stack((x, y, z), axis=-1)

def crop(points, p_keep):
    rand_xyz = uniform_2_sphere()
    centroid = np.mean(points[:, :3], axis=0)
    points_centered = points[:, :3] - centroid

    dist_from_plane = np.dot(points_centered, rand_xyz)
    if p_keep == 0.5:
        mask = dist_from_plane > 0
    else:
        mask = dist_from_plane > np.percentile(dist_from_plane, (1.0 - p_keep) * 100)

    return points[mask, :]

In [8]:
class ModelNet40(torch.utils.data.Dataset):
    def __init__(self, partition='train', crop_type=True, gaussian_noise=False, unseen=False, factor=4):
        self.data, self.label = load_data(partition)
        self.partition = partition
        self.gaussian_noise = gaussian_noise
        self.unseen = unseen
        self.label = self.label.squeeze()
        self.factor = factor
        self.crop_type = crop_type
        if self.unseen:
            if self.partition == 'test':
                self.data = self.data[self.label>=20]
                self.label = self.label[self.label>=20]
            elif self.partition == 'train':
                self.data = self.data[self.label<20]
                self.label = self.label[self.label<20]

    def __getitem__(self, item):
        pointcloud = self.data[item][:1024]
        if self.partition != 'train':
            np.random.seed(item)
        if self.gaussian_noise:
            pointcloud = jitter_pointcloud(pointcloud)
        anglex = np.random.uniform() * np.pi / self.factor
        angley = np.random.uniform() * np.pi / self.factor
        anglez = np.random.uniform() * np.pi / self.factor

        cosx = np.cos(anglex)
        cosy = np.cos(angley)
        cosz = np.cos(anglez)
        sinx = np.sin(anglex)
        siny = np.sin(angley)
        sinz = np.sin(anglez)
        Rx = np.array([[1, 0, 0],
                 [0, cosx, -sinx],
                 [0, sinx, cosx]])
        Ry = np.array([[cosy, 0, siny],
                 [0, 1, 0],
                 [-siny, 0, cosy]])
        Rz = np.array([[cosz, -sinz, 0],
                 [sinz, cosz, 0],
                 [0, 0, 1]])
        R = Rx.dot(Ry).dot(Rz)
        translation = np.array([np.random.uniform(-0.5, 0.5), np.random.uniform(-0.5, 0.5),
                                   np.random.uniform(-0.5, 0.5)])

        scr = pointcloud

        rotation = Rotation.from_euler('zyx', [anglez, angley, anglex])
        tgt = rotation.apply(scr) + translation

        if self.crop_type:
            scr = crop(scr, 0.8)
            tgt = crop(tgt, 0.8)

        return scr.astype('float32'), tgt.astype('float32'), R.astype('float32'), \
               translation.astype('float32')

    def __len__(self):
        return self.data.shape[0]

## 重叠率100%

In [9]:
testdata = ModelNet40(partition='test', crop_type=False, unseen=True)
test_loader = torch.utils.data.DataLoader(
        testdata,
        batch_size=1, shuffle=False, num_workers=0)

In [10]:
rotations_est = []
translations_est = []
rotations_ab = []
translations_ab = []
with torch.no_grad():
    for i, datasets in enumerate(test_loader):
        srcpcd, tgtpcd, rotation_ab, translation_ab = datasets
        est_g, r = act.do_estimate(tgtpcd, srcpcd, model, args.device)
        rotations_est.append(est_g[:, 0:3, 0:3])
        translations_est.append(est_g[:, 0:3, 3])
        rotations_ab.append(rotation_ab)
        translations_ab.append(translation_ab)    

In [11]:
rot_est = np.concatenate(rotations_est, axis=0)
trans_est = np.concatenate(translations_est, axis=0)
rot_ab = np.concatenate(rotations_ab, axis=0)
trans_ab = np.concatenate(translations_ab, axis=0)

In [12]:
test_r_mse = np.mean((rot_ab - rot_est) ** 2)
test_r_rmse = np.sqrt(test_r_mse)
test_r_mae = np.mean(np.abs(rot_ab - rot_est))
test_t_mse = np.mean((trans_ab - trans_est) ** 2)
test_t_rmse = np.sqrt(test_t_mse)
test_t_mae = np.mean(np.abs(trans_ab - trans_est))

In [13]:
message_r = 'test_r_mse:{:.6f},test_r_rmse:{:.6f},test_r_mae:{:.6f}'.format(test_r_mse, test_r_rmse, test_r_mae)
message_t = 'test_t_mse:{:.6f},test_t_rmse:{:.6f},test_t_mae:{:.6f}'.format(test_t_mse, test_t_rmse, test_t_mae)
print(message_r, message_t)

test_r_mse:0.036270,test_r_rmse:0.190447,test_r_mae:0.047239 test_t_mse:0.001458,test_t_rmse:0.038180,test_t_mae:0.007684


## 重叠率<100%

In [9]:
pat_testdata = ModelNet40(partition='test', crop_type=True, gaussian_noise=True, unseen=True)
par_testloader = torch.utils.data.DataLoader(
        pat_testdata,
        batch_size=1, shuffle=False, num_workers=0)

In [13]:
for i, datasets in enumerate(par_testloader):
    srcpcd, tgtpcd, rotation_ab, translation_ab = datasets
    break
srcpcd.shape

torch.Size([1, 819, 3])

In [10]:
rotations_est = []
translations_est = []
rotations_ab = []
translations_ab = []
with torch.no_grad():
    for i, datasets in enumerate(par_testloader):
        srcpcd, tgtpcd, rotation_ab, translation_ab = datasets
        est_g, r = act.do_estimate(tgtpcd, srcpcd, model, args.device)
        rotations_est.append(est_g[:, 0:3, 0:3])
        translations_est.append(est_g[:, 0:3, 3])
        rotations_ab.append(rotation_ab)
        translations_ab.append(translation_ab) 

In [11]:
rot_est = np.concatenate(rotations_est, axis=0)
trans_est = np.concatenate(translations_est, axis=0)
rot_ab = np.concatenate(rotations_ab, axis=0)
trans_ab = np.concatenate(translations_ab, axis=0)

In [12]:
test_r_mse = np.mean((rot_ab - rot_est) ** 2)
test_r_rmse = np.sqrt(test_r_mse)
test_r_mae = np.mean(np.abs(rot_ab - rot_est))
test_t_mse = np.mean((trans_ab - trans_est) ** 2)
test_t_rmse = np.sqrt(test_t_mse)
test_t_mae = np.mean(np.abs(trans_ab - trans_est))

In [13]:
message_r = 'test_r_mse:{:.6f},test_r_rmse:{:.6f},test_r_mae:{:.6f}'.format(test_r_mse, test_r_rmse, test_r_mae)
message_t = 'test_t_mse:{:.6f},test_t_rmse:{:.6f},test_t_mae:{:.6f}'.format(test_t_mse, test_t_rmse, test_t_mae)
print(message_r, message_t)

test_r_mse:0.072054,test_r_rmse:0.268428,test_r_mae:0.177255 test_t_mse:0.012495,test_t_rmse:0.111782,test_t_mae:0.084300


In [92]:
pt_tree = o3d.geometry.KDTreeFlann(ptpcd)
pspcd.transform(G)
count = 0
idxs_pt = []
idxs_ps = []
dists = []
for i in range(len(pspcd.points)):
    #[k, idx, dist] = bun00_tree.search_knn_vector_3d(bun45.points[int(ind1[i])], 3)
    [k, idx, dist] = pt_tree.search_radius_vector_3d(pspcd.points[i], 0.001)
    if k!= 0:
        count += 1
        idxs_pt.append(idx[0])
        idxs_ps.append(i)
        dists.append(dist[0])
count/len(pspcd.points)

0.7496947496947497

In [44]:
ptpcd = o3d.geometry.PointCloud()
ptpcd.points = o3d.utility.Vector3dVector(tgt.reshape(-1, 3))
#ptpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
#ptpcd.orient_normals_to_align_with_direction()
ptpcd.paint_uniform_color([1, 0, 0])

pspcd = o3d.geometry.PointCloud()
pspcd.points = o3d.utility.Vector3dVector(src.reshape(-1, 3))
#pspcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
#pspcd.orient_normals_to_align_with_direction()
pspcd.paint_uniform_color([0, 0, 1])

o3d.visualization.draw_geometries([ptpcd, pspcd])

In [40]:
rot = rotation_ab.numpy().reshape(3, 3)
trans = translation_ab.numpy()
G = np.concatenate((rot, trans.reshape(3, -1)), axis = 1)
G = np.concatenate((G, [[0.,0.,0.,1.]]), axis = 0)

In [17]:
#显示真实对应的点
ptpcd.paint_uniform_color([0.5, 0.5, 0.5])
pspcd.paint_uniform_color([0.5, 0.5, 0.5])
np.asarray(ptpcd.colors)[idxs_pt[0:], :] = [255/255, 127/255, 39/255]
np.asarray(pspcd.colors)[idxs_ps[0:], :] = [160/255, 255/255, 249/255]
o3d.visualization.draw_geometries([ptpcd, pspcd])

In [36]:
try_pt = torch.tensor(tgt).unsqueeze(0).contiguous().to(torch.float32)
try_ps = torch.tensor(src).unsqueeze(0).contiguous().to(torch.float32)
try_pt.shape, try_ps.shape

(torch.Size([1, 1638, 3]), torch.Size([1, 1638, 3]))

### no sampling

In [37]:
with torch.no_grad():
    est_g, r, f_series, feas = act.do_estimate(try_pt, try_ps, model, args.device)

In [38]:
g_series = model.g_series

In [41]:
est_g, g_series[9], G

(tensor([[[ 0.4037, -0.1294,  0.9057,  0.0459],
          [ 0.5931,  0.7908, -0.1514,  0.1883],
          [-0.6966,  0.5983,  0.3960,  0.0151],
          [ 0.0000,  0.0000,  0.0000,  1.0000]]]),
 tensor([[[ 0.7512, -0.1052,  0.6516,  0.1032],
          [ 0.4259,  0.8315, -0.3568,  0.1865],
          [-0.5043,  0.5456,  0.6694,  0.0353],
          [ 0.0000,  0.0000,  0.0000,  1.0000]]]),
 array([[ 0.75326449, -0.38586897,  0.53263289,  0.04488318],
        [ 0.6122871 ,  0.70714998, -0.35361478, -0.0763452 ],
        [-0.24020238,  0.59248966,  0.76893353,  0.14589411],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]))

In [45]:
pspcd.transform(g_series[9].numpy().reshape(4,4))

PointCloud with 1638 points.

In [46]:
o3d.visualization.draw_geometries([ptpcd, pspcd])

In [77]:
o3d.visualization.draw_geometries([ptpcd, est_ptpcd])

In [63]:
ps_points = np.asarray(pspcd.points)
pt_points = np.asarray(ptpcd.points)
centroid_ps = np.mean(ps_points, axis=0).reshape(1,3)
centroid_pt = np.mean(pt_points, axis=0).reshape(1,3)
centroid_ps, centroid_pt

(array([[ 2.38428076e-07, -9.24305869e-08,  2.64178347e-07]]),
 array([[ 0.04488338, -0.07634532,  0.1458944 ]]))

In [65]:
pts = np.concatenate((pt_points, centroid_pt), axis = 0)
pss = np.concatenate((ps_points, centroid_ps), axis = 0)
ptpcd.points = o3d.utility.Vector3dVector(pts.reshape(-1, 3))
pspcd.points = o3d.utility.Vector3dVector(pss.reshape(-1, 3))
np.asarray(pspcd.points)[-1]

array([ 2.38428076e-07, -9.24305869e-08,  2.64178347e-07])

In [66]:
ptpcd.paint_uniform_color([0.5, 0.5, 0.5])
pspcd.paint_uniform_color([0.5, 0.5, 0.5])
ptpcd.colors[-1] = [255/255, 127/255, 39/255]
pspcd.colors[-1] = [160/255, 255/255, 249/255]
o3d.visualization.draw_geometries([ptpcd, pspcd])

In [ ]:
A_F = model.ptnet(centroid_A)
B_F = model.ptnet(centroid_B)

In [60]:
points = np.asarray(ptpcd.points)
points = torch.tensor(points).unsqueeze(0).contiguous().to(torch.float32)
points_F = model.ptnet(points)
points_feas = model.ptnet.features

## imaging

In [56]:
g_hat = est_g.cpu().contiguous().view(-1, 4, 4)
est_trans_pt = ptlk.se3.transform(g_hat.to(try_ps), try_ps.transpose(1,2).contiguous()).transpose(1,2).contiguous()
est_pt = est_trans_pt[0].detach().numpy()
est_pt.shape

NameError: name 'try_ps' is not defined

In [55]:
est_ptpcd = o3d.geometry.PointCloud()
est_ptpcd.points = o3d.utility.Vector3dVector(est_pt.reshape(-1, 3))
#est_ptpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
#est_ptpcd.orient_normals_to_align_with_direction()
est_ptpcd.paint_uniform_color([0, 0, 1])

NameError: name 'est_pt' is not defined

In [54]:
o3d.visualization.draw_geometries([ptpcd, est_ptpcd])

## error

In [23]:
est_rot = est_g[:, 0:3, 0:3].reshape(3, 3).numpy()
est_trans = est_g[:, 0:3, 3].reshape(3, 1).numpy()
gt_rot = G[0:3, 0:3]
gt_trans = G[0:3, 3].reshape(3, 1)

In [24]:
test_r_mse = np.mean((gt_rot - est_rot) ** 2)
test_r_rmse = np.sqrt(test_r_mse)
test_r_mae = np.mean(np.abs(gt_rot - est_rot))
test_t_mse = np.mean((gt_trans - est_trans) ** 2)
test_t_rmse = np.sqrt(test_t_mse)
test_t_mae = np.mean(np.abs(gt_trans - est_trans))

In [25]:
message_r = 'test_r_mse:{:.6f},test_r_rmse:{:.6f},test_r_mae:{:.6f}'.format(test_r_mse, test_r_rmse, test_r_mae)
message_t = 'test_t_mse:{:.6f},test_t_rmse:{:.6f},test_t_mae:{:.6f}'.format(test_t_mse, test_t_rmse, test_t_mae)
print(message_r, message_t)

test_r_mse:0.080324,test_r_rmse:0.283415,test_r_mae:0.235490 test_t_mse:0.026601,test_t_rmse:0.163097,test_t_mae:0.153965


### features

In [23]:
ptfeas, psfeas = feas['pt_feas'], feas['ps_feas']
fea_tree = KDTree(ptfeas[0])

tensor([[0.0000, 0.0000, 0.1645,  ..., 0.0000, 2.3249, 1.8158],
        [0.7780, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.9656, 0.0000, 0.0000,  ..., 0.0000, 1.1518, 1.3219],
        ...,
        [1.1753, 0.4132, 0.0000,  ..., 0.8117, 0.0000, 0.0466],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.7720],
        [1.4908, 1.0571, 0.0000,  ..., 0.0000, 0.5977, 1.1924]])

In [23]:
mo_correspond = []
dists = []
#K = 3
for i, fea in enumerate(psfeas[0]):
    idx = fea_tree.query_radius(fea.reshape(1, -1), r=1)
#    dist, idx = fea_tree.query(fea.reshape(1, -1), k=1) 
    #if K is not None:
    #    idx = idx[0][:K]
    # = tup[0], tup[1]
#    dists.append(dist)
    for j in range(idx[0].shape[-1]):
        mo_correspond.append([i, idx[0][j]])  #, dist[0][j]])
    #for j in idx:
    #    correspondences.append([i, j]) 

In [25]:
mo_correspond = np.array(mo_correspond)
psidx = list(set(mo_correspond[:,0].tolist()))
ptidx = list(set(mo_correspond[:,1].tolist()))
len(ptidx), len(psidx)

(29, 25)

In [26]:
ptpcd.paint_uniform_color([0.5, 0.5, 0.5])
pspcd.paint_uniform_color([0.5, 0.5, 0.5])
np.asarray(ptpcd.colors)[ptidx[0:], :] = [1, 0, 0]
np.asarray(pspcd.colors)[psidx[0:], :] = [0, 0, 1]
o3d.visualization.draw_geometries([ptpcd, pspcd])

In [5]:
def get_datasets(args):

    cinfo = None
    if args.categoryfile:
        #categories = numpy.loadtxt(args.categoryfile, dtype=str, delimiter="\n").tolist()
        categories = [line.rstrip('\n') for line in open(args.categoryfile)]
        categories.sort()
        c_to_idx = {categories[i]: i for i in range(len(categories))}
        cinfo = (categories, c_to_idx)

    perturbations = None
    fmt_trans = False
    if args.perturbations:
        perturbations = np.loadtxt(args.perturbations, delimiter=',')
    if args.format == 'wt':
        fmt_trans = True

    if args.dataset_type == 'modelnet':
        transform = torchvision.transforms.Compose([\
                ptlk.data.transforms.Mesh2Points(),\
                ptlk.data.transforms.OnUnitCube(),\
            ])

        testdata = ptlk.data.datasets.ModelNet(args.dataset_path, train=0, transform=transform, classinfo=cinfo)

        testset = ptlk.data.datasets.CADset4tracking_fixed_perturbation(testdata,\
                        perturbations, fmt_trans=fmt_trans)

    return testset

In [6]:
testset = get_datasets(args)
testloader = torch.utils.data.DataLoader(
    testset,
    batch_size=1, shuffle=False, num_workers=args.workers)

In [7]:
for i, datasets in enumerate(testloader):
    tgtpcd, srcpcd, igt = datasets
    break

In [8]:
tgtpcd.shape, srcpcd.shape, igt.shape

(torch.Size([1, 1187, 3]), torch.Size([1, 1187, 3]), torch.Size([1, 4, 4]))

In [9]:
ptpcd = o3d.geometry.PointCloud()
ptpcd.points = o3d.utility.Vector3dVector(tgtpcd.numpy().reshape(-1, 3))
#ptpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
#ptpcd.orient_normals_to_align_with_direction()
ptpcd.paint_uniform_color([1, 0, 0])

pspcd = o3d.geometry.PointCloud()
pspcd.points = o3d.utility.Vector3dVector(srcpcd.numpy().reshape(-1, 3))
#pspcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
#pspcd.orient_normals_to_align_with_direction()
pspcd.paint_uniform_color([0, 0, 1])

o3d.visualization.draw_geometries([ptpcd, pspcd])

In [10]:
with torch.no_grad():
    est_g, r, f_series, feas = act.do_estimate(tgtpcd, srcpcd, model, args.device)

In [11]:
g_hat = est_g.cpu().contiguous().view(-1, 4, 4)
est_trans_pt = ptlk.se3.transform(g_hat.to(srcpcd), srcpcd.transpose(1,2).contiguous()).transpose(1,2).contiguous()
est_pt = est_trans_pt[0].detach().numpy()
est_pt.shape

(1187, 3)

In [12]:
est_ptpcd = o3d.geometry.PointCloud()
est_ptpcd.points = o3d.utility.Vector3dVector(est_pt.reshape(-1, 3))
#est_ptpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
#est_ptpcd.orient_normals_to_align_with_direction()
est_ptpcd.paint_uniform_color([0, 0, 1])

PointCloud with 1187 points.

In [41]:
o3d.visualization.draw_geometries([ptpcd, est_ptpcd])

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


In [6]:
def unit_cube(vertices):
    pcd = np.array(vertices)
    c = np.max(pcd, axis=0) - np.min(pcd, axis=0) # [N, D] -> [D]
    s = np.max(c) # -> scalar
    v = pcd / s
    return v - v.mean(axis=0)

In [7]:
vertices = []
mopath = r'D:\SDXX\ModelNet40\car\test\car_0209.off'
with open(mopath, 'r') as fin:
    sig = fin.readline().strip()
    line = fin.readline().strip()
    num_verts, num_faces, num_edges = tuple([int(s) for s in line.split(' ')])
    for v in range(num_verts):
        vp = tuple(float(s) for s in fin.readline().strip().split(' '))
        vertices.append(vp)
tgtpcd = unit_cube(vertices)

In [66]:
anglex = np.random.uniform() * np.pi / 4
angley = np.random.uniform() * np.pi / 4
anglez = np.random.uniform() * np.pi / 4

cosx = np.cos(anglex)
cosy = np.cos(angley)
cosz = np.cos(anglez)
sinx = np.sin(anglex)
siny = np.sin(angley)
sinz = np.sin(anglez)
Rx = np.array([[1, 0, 0],
         [0, cosx, -sinx],
         [0, sinx, cosx]])
Ry = np.array([[cosy, 0, siny],
         [0, 1, 0],
         [-siny, 0, cosy]])
Rz = np.array([[cosz, -sinz, 0],
         [sinz, cosz, 0],
         [0, 0, 1]])
R = Rx.dot(Ry).dot(Rz)
translation = np.array([np.random.uniform(0, 0.1), np.random.uniform(0, 0.1),
                                   np.random.uniform(0, 0.1)])

rotation = Rotation.from_euler('zyx', [anglez, angley, anglex])
srcpcd = rotation.apply(tgtpcd) + translation

src = crop(srcpcd, 0.9)
tgt = crop(tgtpcd, 0.9)

In [81]:
def writef(fname_pt, fname_ps, pt, ps):
    with open(fname_pt, 'w') as fout:
        print('%d'%pt.shape[0],file=fout)
        for i in range(pt.shape[0]):
            print(' '.join(map(str, pt[i])),file=fout)   #变换后的源：绿色
            
    with open(fname_ps, 'w') as fout:
        print('%d'%ps.shape[0],file=fout)
        for i in range(ps.shape[0]):
            print(' '.join(map(str, ps[i])),file=fout)  #模板：黄色
            
fname_t = r'C:\Users\fcky\Desktop\3D\DaiMa\EBABC_Kuang/car_pt.asc'
fname_s = r'C:\Users\fcky\Desktop\3D\DaiMa\EBABC_Kuang/car_ps.asc'

writef(fname_t, fname_s, tgt, src)

In [67]:
ptpcd = o3d.geometry.PointCloud()
ptpcd.points = o3d.utility.Vector3dVector(tgt.reshape(-1, 3))
#ptpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
#ptpcd.orient_normals_to_align_with_direction()
ptpcd.paint_uniform_color([255/255, 127/255, 39/255])

pspcd = o3d.geometry.PointCloud()
pspcd.points = o3d.utility.Vector3dVector(src.reshape(-1, 3))
#pspcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
#pspcd.orient_normals_to_align_with_direction()
pspcd.paint_uniform_color([160/255, 255/255, 249/255])

o3d.visualization.draw_geometries([ptpcd, pspcd])

In [68]:
rot = R.T
trans = -rot.dot(translation)
G = np.concatenate((rot, trans.reshape(3, -1)), axis = 1)
G = np.concatenate((G, [[0.,0.,0.,1.]]), axis = 0)

In [69]:
try_pt = torch.tensor(tgt).unsqueeze(0).contiguous().to(torch.float32)
try_ps = torch.tensor(src).unsqueeze(0).contiguous().to(torch.float32)
try_pt.shape, try_ps.shape

(torch.Size([1, 50077, 3]), torch.Size([1, 50079, 3]))

In [70]:
pt_kwy = int(tgt.shape[0] / 1024)
pt_num = np.random.randint(0, pt_kwy, 1)
tgtidx = list(range(pt_num[0], tgt.shape[0], int(pt_kwy+1)))
ps_kwy = int(src.shape[0] / 1024)
ps_num = np.random.randint(0, ps_kwy, 1)
scridx = list(range(ps_num[0], src.shape[0], int(ps_kwy+1)))

In [71]:
sampt = try_pt[:, tgtidx, :]
samps = try_ps[:, scridx, :]
with torch.no_grad():
    est_g, r = act.do_estimate(sampt, samps, model, args.device)

In [72]:
g_hat = est_g.cpu().contiguous().view(-1, 4, 4)
est_trans_pt = ptlk.se3.transform(g_hat.to(try_ps), try_ps.transpose(1,2).contiguous()).transpose(1,2).contiguous()
est_pt = est_trans_pt[0].detach().numpy()
est_pt.shape

(50079, 3)

In [73]:
est_ptpcd = o3d.geometry.PointCloud()
est_ptpcd.points = o3d.utility.Vector3dVector(est_pt.reshape(-1, 3))
est_ptpcd.paint_uniform_color([160/255, 255/255, 249/255])

PointCloud with 50079 points.

In [74]:
o3d.visualization.draw_geometries([ptpcd, est_ptpcd])

In [75]:
est_rot = est_g[:, 0:3, 0:3].reshape(3, 3).numpy()
est_trans = est_g[:, 0:3, 3].reshape(3, 1).numpy()
gt_rot = G[0:3, 0:3]
gt_trans = G[0:3, 3].reshape(3, 1)

In [76]:
test_r_mse = np.mean((gt_rot - est_rot) ** 2)
test_r_rmse = np.sqrt(test_r_mse)
test_r_mae = np.mean(np.abs(gt_rot - est_rot))
test_t_mse = np.mean((gt_trans - est_trans) ** 2)
test_t_rmse = np.sqrt(test_t_mse)
test_t_mae = np.mean(np.abs(gt_trans - est_trans))

In [77]:
message_r = 'test_r_mse:{:.6f},test_r_rmse:{:.6f},test_r_mae:{:.6f}'.format(test_r_mse, test_r_rmse, test_r_mae)
message_t = 'test_t_mse:{:.6f},test_t_rmse:{:.6f},test_t_mae:{:.6f}'.format(test_t_mse, test_t_rmse, test_t_mae)
print(message_r, message_t)

test_r_mse:0.003408,test_r_rmse:0.058378,test_r_mae:0.042210 test_t_mse:0.000472,test_t_rmse:0.021717,test_t_mae:0.017952
